Semi-supervised surgery pipeline with SEMAFOVI and SCANVI.
Changes and adaptations are made to the original pipeline in scArches created by:

Lotfollahi, M., Naghipourfar, M., Luecken, M. D., Khajavi,
M., B  ̈uttner, M., Wagenstetter, M., Avsec, ˇZ., Gayoso,
A., Yosef, N., Interlandi, M., et al. Mapping single-cell
data to reference atlases by transfer learning. Nature
Biotechnology, pp. 1–10, 2021

For the original pipeline information, see: https://docs.scarches.org/en/latest/scanvi_surgery_pipeline.html#

In [ ]:
try:
    from nbproject import header
    header()
except ModuleNotFoundError:
    print("If you want to see the header with dependencies, please install nbproject - pip install nbproject")

In [ ]:
import os
os.chdir('../')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
import sys
sys.path.append('path of the modified scvi project')
from scvi.model import SCVI
from scvi.model import SCANVI
from scvi.model import SEMAFOVI

In [ ]:
import scanpy as sc
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import numpy as np
import gdown

In [ ]:
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))
torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)

Set relevant anndata.obs labels and training length

In this pipeline the pancreas scRNA-seq data were utilized to explain the training process, the same procedure also works for the bone marrow human cell atlas scRNA-seq data or any kind of scRNA-seq data in similar structure to the two datasets mentioned above.
For the bone marrow human cell atlas scRNA-seq data, please see:
https://openreview.net/forum?id=gN35BGa1Rt

In [ ]:
condition_key = 'study'
cell_type_key = 'cell_type'
target_conditions = ['Pancreas CelSeq2','Pancreas inDrop', 'Pancreas SS2']

In [ ]:
import anndata as ad

In [ ]:
url = 'https://drive.google.com/uc?id=1ehxgfHTsMZXy6YzlFKGJOsBKQ5rrvMnd' #this is provided in the original pipeline
output = 'pancreas.h5ad'
gdown.download(url, output, quiet=False)

In [ ]:
adata_all = ad.read_h5ad('pancreas.h5ad')

In [ ]:
adata_all.obs.study.unique().tolist()

In [ ]:
pancreas_inDrop = adata_all[adata_all.obs['study'] == 'Pancreas inDrop']
pancreas_CelSeq2 = adata_all[adata_all.obs['study'] == 'Pancreas CelSeq2']
pancreas_CelSeq = adata_all[adata_all.obs['study'] == 'Pancreas CelSeq']
pancreas_Fluidigm_C1 = adata_all[adata_all.obs['study'] == 'Pancreas Fluidigm C1']
pancreas_SS2 = adata_all[adata_all.obs['study'] == 'Pancreas SS2']

In [ ]:
num_observations = pancreas_inDrop.n_obs
print(f"Number of observations for 'pancreas_inDrop': {num_observations}")
num_observations = pancreas_CelSeq2.n_obs
print(f"Number of observations for 'pancreas_CelSeq2': {num_observations}")
num_observations = pancreas_CelSeq.n_obs
print(f"Number of observations for 'pancreas_CelSeq': {num_observations}")
num_observations = pancreas_Fluidigm_C1.n_obs
print(f"Number of observations for 'pancreas_Fluidigm C1': {num_observations}")
num_observations = pancreas_SS2.n_obs
print(f"Number of observations for 'pancreas_SS2': {num_observations}")

In [ ]:
adata = adata_all.raw.to_adata()
adata = remove_sparsity(adata)
source_adata = adata[~adata.obs[condition_key].isin(target_conditions)].copy()
target_adata = adata[adata.obs[condition_key].isin(target_conditions)].copy()

In [ ]:
source_adata

In [ ]:
target_adata

Create SCANVI/SEMAFOVI model and train it on fully labelled reference dataset

In [ ]:
SCVI.setup_anndata(source_adata, batch_key=condition_key, labels_key=cell_type_key)

In [ ]:
vae = SCVI(
    source_adata,
    n_layers=2,
    encode_covariates=True,
    deeply_inject_covariates=False,
    use_layer_norm="both",
    use_batch_norm="none",
)

In [ ]:
vae.train()

In [ ]:
vae.save("path for saved models", overwrite=True)

In [ ]:
loaded_vae = SCVI.load("path for saved models", adata = source_adata)

Choose VAE to initialize

In [ ]:
scanvae = SCANVI.from_scvi_model(loaded_vae, unlabeled_category = "Unknown")

In [ ]:
semafovae = SEMAFOVI.from_scvi_model(loaded_vae, unlabeled_category = "Unknown")

In [ ]:
print("Labelled Indices: ", len(semafovae._labeled_indices)) # or scanvae
print("Unlabelled Indices: ", len(semafovae._unlabeled_indices))

Choose VAE to train

In [ ]:
scanvae.train(max_epochs=20)

In [ ]:
semafovae.train(max_epochs=20)

Create anndata file of latent representation and compute UMAP

In [ ]:
reference_latent = sc.AnnData(scanvae.get_latent_representation()) # or scanvae
reference_latent.obs["cell_type"] = source_adata.obs[cell_type_key].tolist()
reference_latent.obs["batch"] = source_adata.obs[condition_key].tolist()

In [ ]:
reference_latent.obs['predictions'] = scanvae.predict() # or scanvae
print("Acc: {}".format(np.mean(reference_latent.obs.predictions == reference_latent.obs.cell_type)))

Perform surgery on reference model and train on query dataset without cell type labels

In [ ]:
model = SCANVI.load_query_data(  # SCANVI/SEMAFOVI
    target_adata,
    scanvae,  #  scanvae/semafovae
    freeze_dropout = True,
)
#adjust the labeld and unlabeled groups in here.
model._unlabeled_indices = np.where(target_adata.obs[condition_key] == 'Pancreas CelSeq2')[0]
model._labeled_indices = np.where((target_adata.obs[condition_key] == 'Pancreas inDrop')| (target_adata.obs[condition_key] == 'Pancreas SS2'))[0]  
#model._unlabeled_indices = np.arange(target_adata.n_obs)
#model._labeled_indices = []
print("Labelled Indices: ", len(model._labeled_indices))
print("Unlabelled Indices: ", len(model._unlabeled_indices))

In [ ]:
model.train(
    max_epochs=100,
    plan_kwargs=dict(weight_decay=0.0),
    check_val_every_n_epoch=10,
)

In [ ]:
query_latent = sc.AnnData(model.get_latent_representation())
query_latent.obs['cell_type'] = target_adata.obs[cell_type_key].tolist()
query_latent.obs['batch'] = target_adata.obs[condition_key].tolist()

In [ ]:
query_latent.obs['predictions'] = model.predict()
print("Acc: {}".format(np.mean(query_latent.obs.predictions == query_latent.obs.cell_type)))

In [ ]:
sc.pp.neighbors(query_latent)
sc.tl.leiden(query_latent)
sc.tl.umap(query_latent)
plt.figure()
sc.pl.umap(
    query_latent,
    color=["cell_type", "predictions"],
    frameon=False,
    wspace=0.6,
)

compute the other numerical measurement metrics

In [ ]:
from sklearn.metrics import (
f1_score, cohen_kappa_score, balanced_accuracy_score
)

# true labels and predicted labels
true_labels = query_latent.obs.cell_type
predicted_labels = query_latent.obs['predictions']

# Calculate F1 score for multi-class classification
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate Cohen's Kappa
kappa = cohen_kappa_score(true_labels, predicted_labels)

# Calculate Balanced Accuracy
balanced_acc = balanced_accuracy_score(true_labels, predicted_labels)

# Print the results
print(f"F1 Score: {f1}")
print(f"Cohen’s Kappa: {kappa}")
print(f"Balanced Accuracy: {balanced_acc}")

Compute Accuracy of model classifier for query dataset and compare predicted and observed cell types

In [ ]:
df = query_latent.obs.groupby(["cell_type", "predictions"]).size().unstack(fill_value=0)
norm_df = df / df.sum(axis=0)

plt.figure(figsize=(8, 8))
_ = plt.pcolor(norm_df)
_ = plt.xticks(np.arange(0.5, len(df.columns), 1), df.columns, rotation=90)
_ = plt.yticks(np.arange(0.5, len(df.index), 1), df.index)
plt.xlabel("Predicted")
plt.ylabel("Observed")
plt.tight_layout()
plt.colorbar()
# plt.savefig('path where the plotted picture needs to be saved')